# Inferentia2 (inf2.48xlarge)에서 배치성 추론 

---

아래의 내용은 [vLLM Project Git Repo](https://github.com/vllm-project/vllm) 의 [examples/offline_inference_neuron.py](https://github.com/vllm-project/vllm/blob/main/examples/offline_inference_neuron.py) 를 노트북에서 실행한 예시 입니다.
        



# 1. 사전 필수 단계
- 아래를 클릭하셔서 사전 단계를 수행 하세요.
    - [AWS Inferentia2 설치 및 실행 가이드](Readme.md)

# 2. 배치 추론 실행

In [ ]:
from vllm import LLM , SamplingParams

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)


In [8]:
llm = LLM(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_num_seqs=8,
    # The max_model_len and block_size arguments are required to be same as
    # max sequence length when targeting neuron device.
    # Currently, this is a known limitation in continuous batching support
    # in transformers-neuronx.
    # TODO(liangfu): Support paged-attention in transformers-neuronx.
    max_model_len=128,
    block_size=128,
    # The device can be automatically detected when AWS Neuron SDK is installed.
    # The device argument can be either unspecified for automated detection,
    # or explicitly assigned.
    device="neuron",
    tensor_parallel_size=2)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

INFO 03-30 13:09:50 llm_engine.py:87] Initializing an LLM engine with config: model='TinyLlama/TinyLlama-1.1B-Chat-v1.0', tokenizer='TinyLlama/TinyLlama-1.1B-Chat-v1.0', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=128, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cpu, seed=0)


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

2024-03-30 13:11:15.000181:  16560  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2024-03-30 13:11:15.000236:  16560  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --target=trn1 --framework=XLA /tmp/ubuntu/neuroncc_compile_workdir/8a62b422-e336-428c-a9ac-a1bc44bc4570/model.MODULE_fa4f5a5bb5043faa77ed+2c2d707e.hlo_module.pb --output /tmp/ubuntu/neuroncc_compile_workdir/8a62b422-e336-428c-a9ac-a1bc44bc4570/model.MODULE_fa4f5a5bb5043faa77ed+2c2d707e.neff --model-type=transformer --auto-cast=none --verbose=35
2024-03-30 13:11:15.000286:  16561  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2024-03-30 13:11:15.000337:  16561  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --target=trn1 --framework=XLA /tmp/ubuntu/neuroncc_compile_workdir/2b4fdd93-3212-4caa-967f-aa6cf57f6441/model.MODULE_55da091ed2d10512034d+2c2d707e.hlo_module.pb --output /tmp/ubuntu/neuroncc_compile_workdir/2b4fdd93-3212-

In [9]:
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s]

2024-Mar-30 13:14:19.0781 13181:17289 [1] nccl_net_ofi_init:1415 CCOM WARN NET/OFI aws-ofi-nccl initialization failed
2024-Mar-30 13:14:19.0782 13181:17289 [1] init.cc:137 CCOM WARN OFI plugin initNet() failed is EFA enabled?


Processed prompts: 100%|██████████| 4/4 [00:00<00:00,  4.16it/s]

Prompt: 'Hello, my name is', Generated text: ' John Smith. How are you? How are you doing? I’m fine'
Prompt: 'The president of the United States is', Generated text: ' furious with Australia after the country refused to hand over asylum seekers'
Prompt: 'The capital of France is', Generated text: ' Paris, with an area of 2,494.92 square'
Prompt: 'The future of AI is', Generated text: ' in machine learning\nThe Future of AI is in machine learning. It is'
